# Курс "Рекомендательные системы"

## Практическое задание урока 3. Коллаборативная фильтрация

### Теоретическая часть

#### Задание 1

Вспомним прошлый вебинар, мы рассматривали User-User рекомендации и Item-Item рекомендации. Чем они отличаются и чем они похожи? Если есть функция item_item_rec(interaction_matrix). Можно ли использовать эту функцию для user_user_rec?  
В чем принципиальные отличия item-item рекомендаций от ALS?

**Ответ:**

***Чем они отличаются и чем они похожи?***
 - User-User рекомендации  - ищем похожих пользователей. Это называется «рекомендации, основанные на пользователях» (user-based collaborative filtering).
 - Item-Item рекомендации - ищем похожие продукты. Это называется «рекомендации, основанные на продуктах» (item-based collaborative filtering).
  
  Основной алгоритм User-User рекомендаций и Item-Item рекомендаций похож:
1. Найти, насколько другие пользователи (продукты) в базе данных похожи на данного пользователя (продукт).
2. По оценкам других пользователей (продуктов) предсказать, какую оценку даст данный пользователь данному продукту, учитывая с большим весом тех пользователей (продукты), которые больше похожи на данный.


***Если есть функция item_item_rec(interaction_matrix). Можно ли использовать эту функцию для user_user_rec?***
   - Да, можно использовать. Для это слелует транспонировать матрицу interaction_matrix

***В чем принципиальные отличия item-item рекомендаций от ALS?***
- В основу item-item рекомендаций заложен алгоритм KNN, а в ALS - алгоритм градиентного спуска.
- В ALS используется факт, если даже пользователь не имел итеракции с товаром. В item-item используются только факты интеракции.

**Задание 2**

Приведите 3 примера весов (те, которых не было на вебинаре: сумма покупок, количество покупок - неинтересно) user-item матрицы для задачи рекомендаций товаров 

**Ответ:**

- Количество товара купленного покупателем из той же группы товаров, к которому относится интересующий товар
- Средний чек покапателя
- Частота покупок
- Частота покупки * цена
- Стоимость проданного товара

**Задание 3**

Какие ограничения есть у ALS? (Тип информации, линейность/нелинейность факторов и т д)

**Ответ**

- Линейность. ALS может только линейно искать взаимодейстиве между пользователем и товаром.
- ALS не использует никакую информацию о товаре или пользователе, а строит свои рекомендации лишь на истории оценок.

**Задание 4**

Мы рассматривали bm_25_weight. 
Опишите, как он работает. Как сделать рекомендации только на основе bm_25? (Можно и нужно пользоваться любыми источниками, приложите на них ссылки). Какие еще способы перевзвешивания user-item матрицы Вы знаете / можете предложить (ещё 2-3 способа)?

**Ответ**

***Мы рассматривали bm_25_weight. Опишите, как он работает.***
- Алгоритм bm_25_weight взвешивает каждую отдельную строку user-item матрицы.
- Используется зависимость релевантности от вхождения или не вхождения слов в запросах с более чем одного слова. 
- Релевантность всего поискового запроса равна сумме релевантностей всех слов. 
- Отсутствие слова дает релевантность 0.

***Как сделать рекомендации только на основе bm_25? (Можно и нужно пользоваться любыми источниками, приложите на них ссылки)***

***Какие еще способы перевзвешивания user-item матрицы Вы знаете / можете предложить (ещё 2-3 способа)?***



### Практическая часть

In [48]:
import implicit

In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from pathlib import Path

In [50]:
def precision_at_k(recommended_list, bought_list, k=5):
    """Функция для расчета Precision@k"""
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [52]:
DATA_ROOT = Path('./data/')

# input
DATASET_PATH = DATA_ROOT / 'retail_train.csv'
ITEM_PATH = DATA_ROOT / 'product.csv'

In [53]:
data = pd.read_csv(DATASET_PATH)
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [54]:
# возьмем последние 3 недели в качестве теста
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [55]:
item_features = pd.read_csv(ITEM_PATH)
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [56]:
# item_features.department.nunique()

print('Количество категорий:', item_features.department.nunique())

Количество категорий: 44


In [57]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


Обучим модель на топ-5000 товаров

In [58]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [59]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
user_item_matrix.shape

(2499, 5001)

In [61]:
# перенумеруем пользователей и товары
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

### Базовая модель ALS (из вебинара)

In [63]:
%%time

model = AlternatingLeastSquares(factors=100, #k
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)


Wall time: 6.02 s


In [64]:
recs

[(4016, 1.0621603982960965),
 (3688, 1.0556903524742438),
 (2381, 0.9945976079651067),
 (4157, 0.939316468647643),
 (3408, 0.9356971888253919)]

In [65]:
[id_to_itemid[rec[0]] for rec in recs]

[1133018, 1106523, 999999, 5569230, 1082185]

In [66]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [67]:
result

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
...,...,...
2037,2496,[6534178]
2038,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ..."
2039,2498,"[15716530, 834484, 901776, 914190, 958382, 972..."
2040,2499,"[867188, 877580, 902396, 914190, 951590, 95813..."


In [68]:
%%time
    
result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

Wall time: 1min 5s


In [69]:
result.head(2)

,user_id,actual,als
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1033142, 5569374, 995242, 979707, 999999]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[965766, 1106523, 1133018, 5569327, 5568378]"


In [70]:
prec_als = result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
print('Precision@5 по ALS:', '%.3f' % prec_als)

Precision@5 for ALS: 0.162


### BM25 взвешивание (из вебинара)

In [71]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [73]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4) # K - кол-во ближайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


Wall time: 5.45 s


In [74]:
%%time

result['als_bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

In [75]:
prec_bm25 = result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean()
print('Precision@5 по BM25:', '%.3f' % prec_bm25)

Precision@5 for BM25: 0.192


### Задание 5. Подбор матрицы $c_{ui}$
Попробуйте различные варианты матрицы весов (3+ вариантов). 

Обучите алгоритм для различных $C$. 

В качестве результата приведите таблицу: матрица весов - результат на train и validation.

Сделате качественные выводы.

#### Первый вариант весов: aggfunc='nunique'

In [76]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

# заново создаём user-item матрицу, чтобы не применять 2 взвешивания подряд.
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='nunique', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице !

In [78]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=8) # K - кол-во ближайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


Wall time: 3.49 s


In [79]:
%%time

result['als_bm25_nunique'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

Wall time: 1min 58s


In [80]:
prec_nunique = result.apply(lambda row: precision_at_k(row['als_bm25_nunique'], row['actual']), axis=1).mean()
print('Precision@5 по BM25 с весом nunique:', '%.3f' % prec_nunique)

Precision@5 по BM25 с весом nunique: 0.183


#### Второй вариант весов: aggfunc='mean'

In [81]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

# заново создаём user-item матрицу, чтобы не применять 2 взвешивания подряд.
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='mean', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице !

In [83]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=8) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


Wall time: 3.89 s


In [84]:
%%time

result['als_bm25_mean'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

Wall time: 56.5 s


In [85]:
prec_mean = result.apply(lambda row: precision_at_k(row['als_bm25_mean'], row['actual']), axis=1).mean()
print('Precision@5 по BM25 с весом mean:', '%.3f' % prec_mean)

Precision@5 по BM25 с весом mean: 0.181


#### Третий вариант весов: aggfunc='sum'

In [86]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

# заново создаём user-item матрицу, чтобы не применять 2 взвешивания подряд.
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='sum', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [87]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице !

In [88]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=8) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


Wall time: 3.96 s


In [89]:
%%time

result['als_bm25_quantity'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

Wall time: 58 s


In [90]:
prec_sum = result.apply(lambda row: precision_at_k(row['als_bm25_quantity'], row['actual']), axis=1).mean()
print('Precision@5 по BM25 с весом sum:', '%.3f' % prec_sum)

Precision@5 по BM25 с весом sum: 0.155


#### Четвертый вариант весов: aggfunc='size'

In [91]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

# заново создаём user-item матрицу, чтобы не применять 2 взвешивания подряд.
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='size', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице !

In [93]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=8) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


Wall time: 3.61 s


In [94]:
%%time

result['als_bm25_size'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

Wall time: 57.4 s


In [ ]:
prec_size = result.apply(lambda row: precision_at_k(row['als_bm25_size'], row['actual']), axis=1).mean()
print('Precision@5 по BM25 с весом sum:', '%.3f' % prec_size)

In [96]:
result.head(5)

,user_id,actual,als,als_bm25,als_bm25_nunique,als_bm25_mean,als_bm25_quantity,als_bm25_size
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1033142, 5569374, 995242, 979707, 999999]","[999999, 1100972, 1082185, 995242, 1051211]","[999999, 1082185, 1135694, 8090541, 958046]","[999999, 1082185, 1101010, 1051211, 986912]","[999999, 1082185, 995242, 6534178, 866211]","[999999, 1082185, 995242, 965766, 934369]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[965766, 1106523, 1133018, 5569327, 5568378]","[999999, 1133018, 951590, 1092026, 1053690]","[951590, 826249, 999999, 9337581, 910032]","[999999, 1082185, 6534178, 1029743, 1106523]","[999999, 6534178, 1029743, 1106523, 883404]","[999999, 1053690, 1092026, 951590, 1133018]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1051516, 866211, 930118, 1023720, 1007195]","[1051516, 1082185, 1023720, 1024306, 999999]","[1082185, 999999, 878996, 1024306, 1023720]","[999999, 1082185, 6534178, 1127831, 866211]","[999999, 1023720, 1082185, 1127831, 930118]","[1023720, 1082185, 1024306, 1051516, 895268]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[999999, 1082185, 938700, 1133018, 1013321]","[999999, 1082185, 6944571, 1058997, 826249]","[1123086, 1127831, 857390, 999999, 879635]","[999999, 1082185, 857390, 6534178, 1029743]","[999999, 1082185, 6534178, 995242, 1029743]","[999999, 1082185, 6944571, 895540, 1019247]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[938700, 1004906, 1053690, 840361, 6534178]","[999999, 844179, 840361, 1068719, 965766]","[999999, 1082185, 1068719, 840361, 1004906]","[999999, 1082185, 995242, 6534178, 1029743]","[999999, 995242, 1082185, 6534178, 1029743]","[999999, 883404, 1068719, 1004906, 840361]"


In [98]:
df_res = pd.DataFrame(columns=['matrix', 'precision_at_k'])
i = 0
for column in result.columns[2:]:
    a = round(result.apply(lambda row: precision_at_k(row[column], row['actual']), axis=1).mean(), 3)
    df_res.loc[i] = [column, a]
    i += 1

In [100]:
df_res.sort_values(by='precision_at_k', ascending=False)

,parameters,presision_at_k
5,als_bm25_size,0.193
1,als_bm25,0.192
2,als_bm25_nunique,0.183
3,als_bm25_mean,0.181
0,als,0.162
4,als_bm25_quantity,0.155


Лучшая метрика Precision@5 у матрицы весов по aggfunc='size'.

### Задание 6. Оптимизация гиперпараметров
Для лучшей матрицы весов из первого задания подберите оптимальные $\lambda$ и n_factors. Подбор можно делать вручную (цикл в цикле, аналог sklearn.GridSearch, или случайно - sklearn.RandomSearch). Или Вы можете воспользоваться библиотеками для автоматического подбора гипперпараметров (любые на Ваш вкус). В качестве результата постройте графики:
1. Значение параметра - время обучения 
2. Значение параметра - качество train, качество validation  

Сделайте качественные выводы

In [101]:
%%time

for factors in [50, 100, 300]:
    for reg in [0.0001, 0.001, 0.01]:

        # Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
        data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

        # заново создаём user-item матрицу, чтобы не применять 2 взвешивания подряд.
        user_item_matrix = pd.pivot_table(data_train, 
                                          index='user_id', columns='item_id', 
                                          values='quantity', # Можно пробовать другие варианты
                                          aggfunc='size', 
                                          fill_value=0
                                         )

        user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

        # переведем в формат saprse matrix
        sparse_user_item = csr_matrix(user_item_matrix).tocsr()

        user_item_matrix.head(3)

        user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

        

        model = AlternatingLeastSquares(factors=int(factors), 
                                        regularization=reg,
                                        iterations=15, 
                                        calculate_training_loss=True, 
                                        num_threads=8) # K - кол-во билжайших соседей

        model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
                  show_progress=False)

        result[f'als_bm25_factors={factors}_reg={reg}'] = \
                        result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

        result.apply(lambda row: precision_at_k(row[f'als_bm25_factors={factors}_reg={reg}'], \
                                                            row['actual']), axis=1).mean()

C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

Wall time: 18min 16s


In [102]:
result.head(5)

,user_id,actual,als,als_bm25,als_bm25_nunique,als_bm25_mean,als_bm25_quantity,als_bm25_size,als_bm25_factors=50_reg=0.0001,als_bm25_factors=50_reg=0.001,als_bm25_factors=50_reg=0.01,als_bm25_factors=100_reg=0.0001,als_bm25_factors=100_reg=0.001,als_bm25_factors=100_reg=0.01,als_bm25_factors=300_reg=0.0001,als_bm25_factors=300_reg=0.001,als_bm25_factors=300_reg=0.01
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1033142, 5569374, 995242, 979707, 999999]","[999999, 1100972, 1082185, 995242, 1051211]","[999999, 1082185, 1135694, 8090541, 958046]","[999999, 1082185, 1101010, 1051211, 986912]","[999999, 1082185, 995242, 6534178, 866211]","[999999, 1082185, 995242, 965766, 934369]","[999999, 1082185, 995242, 1100972, 5569374]","[999999, 1082185, 1100972, 1051211, 995242]","[999999, 1082185, 1100972, 995242, 5569374]","[999999, 1082185, 995242, 965766, 824546]","[999999, 1082185, 995242, 1051211, 965766]","[1082185, 999999, 995242, 965766, 934369]","[1082185, 999999, 995242, 1033142, 1082212]","[1105488, 999999, 1082185, 1033142, 995242]","[1082185, 999999, 995242, 1033142, 9527290]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[965766, 1106523, 1133018, 5569327, 5568378]","[999999, 1133018, 951590, 1092026, 1053690]","[951590, 826249, 999999, 9337581, 910032]","[999999, 1082185, 6534178, 1029743, 1106523]","[999999, 6534178, 1029743, 1106523, 883404]","[999999, 1053690, 1092026, 951590, 1133018]","[999999, 951590, 5569230, 1053690, 826249]","[999999, 951590, 844165, 1106523, 1053690]","[999999, 951590, 844165, 826249, 1029743]","[951590, 999999, 1053690, 1092026, 1133018]","[999999, 1133018, 1053690, 1106523, 951590]","[999999, 1053690, 951590, 1092026, 1106523]","[1133018, 1022003, 1106523, 883404, 1092026]","[1133018, 1022003, 883404, 1106523, 999999]","[1133018, 1022003, 883404, 1106523, 1098066]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1051516, 866211, 930118, 1023720, 1007195]","[1051516, 1082185, 1023720, 1024306, 999999]","[1082185, 999999, 878996, 1024306, 1023720]","[999999, 1082185, 6534178, 1127831, 866211]","[999999, 1023720, 1082185, 1127831, 930118]","[1023720, 1082185, 1024306, 1051516, 895268]","[1082185, 1024306, 857006, 911878, 999999]","[1082185, 1024306, 878996, 857006, 999999]","[1082185, 1024306, 1007195, 999999, 1023720]","[1082185, 1024306, 1051516, 1023720, 999999]","[1023720, 1051516, 1082185, 1024306, 999999]","[1082185, 878996, 1024306, 1051516, 857006]","[866211, 878996, 1007195, 1082185, 999999]","[866211, 878996, 1007195, 962568, 1082185]","[866211, 986947, 878996, 1082185, 999999]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[999999, 1082185, 938700, 1133018, 1013321]","[999999, 1082185, 6944571, 1058997, 826249]","[1123086, 1127831, 857390, 999999, 879635]","[999999, 1082185, 857390, 6534178, 1029743]","[999999, 1082185, 6534178, 995242, 1029743]","[999999, 1082185, 6944571, 895540, 1019247]","[999999, 1082185, 6944571, 1058997, 1126899]","[999999, 1082185, 6944571, 849843, 1126899]","[999999, 1082185, 849843, 6944571, 1058997]","[999999, 1082185, 6944571, 893018, 6534178]","[999999, 1082185, 6944571, 1058997, 849843]","[999999, 1082185, 1058997, 6944571, 1126899]","[999999, 1082185, 828867, 1122358, 6944571]","[1082185, 999999, 1122358, 828867, 938700]","[1082185, 999999, 826249, 828867, 1122358]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[938700, 1004906, 1053690, 840361, 6534178]","[999999, 844179, 840361, 1068719, 965766]","[999999, 1082185, 1068719, 840361, 1004906]","[999999, 1082185, 995242, 6534178, 1029743]","[999999, 995242, 1082185, 6534178, 1029743]","[999999, 883404, 1068719, 1004906, 840361]","[999999, 840361, 1082185, 965766, 883404]","[999999, 883404, 840361, 1082185, 826249]","[999999, 1082185, 1004906, 840361, 1033142]","[999999, 883404, 965766, 1053690, 840361]","[999999, 840361, 883404, 1053690, 913785]","[999999, 840361, 1053690, 989075, 1004906]","[833715, 6534178, 1004906, 938700, 913785]","[1004906, 6534178, 

In [103]:
df_param = pd.DataFrame(columns=['parameters', 'precision_at_k'])
i = 0
for column in result.columns[2:]:
    a = round(result.apply(lambda row: precision_at_k(row[column], row['actual']), axis=1).mean(), 3)
    #print(column, a)
    #df = pd.DataFrame([column, a])
    df_param.loc[i] = [column, a]
    i += 1

In [104]:
df_param.sort_values(by='precision_at_k', ascending=False).head(10)

,parameters,precision_at_k
10,als_bm25_factors=100_reg=0.001,0.194
5,als_bm25_size,0.193
11,als_bm25_factors=100_reg=0.01,0.193
1,als_bm25,0.192
9,als_bm25_factors=100_reg=0.0001,0.190
14,als_bm25_factors=300_reg=0.01,0.189
2,als_bm25_nunique,0.183
12,als_bm25_factors=300_reg=0.0001,0.183
13,als_bm25_factors=300_reg=0.001,0.182
3,als_bm25_mean,0.181
